## Turnover Decisions
- Every well phenotype has a chance to be turned overed
- For every transfer there is a unique distribution of turnover strains.
- We count the per phenotype rate of wells that will be discharged at the end of the transfer after the phenotyping
- We also count the turnover distribution at the beginning of each transfer.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import math
from evaluation_scripts.base import get_pathes, dict_to_json, load_toml
from evaluation_scripts.well_ident import build_dictionaries
from itertools import product as iterprod
import json
from model_scripts.decisions_04 import get_tau_out

experiments = ["20220412", "20220127", "20210417"]
regular_phenos = ["UI", "S", "A_r", "B_r", "A&B", "AB_r"] 


In [2]:
for exp in experiments:
    print(exp)
    ## LOAD
    jupyter_path, base_path, path, analysis_path, os = get_pathes(exp + "_output")
    hue_infos, strategies, plate_rows, agar_strain_dict = build_dictionaries()
    Data = pd.read_pickle(analysis_path+os.sep+"obj"+os.sep+"01b_Data.pkl")
    with open(os.path.join(path, "exp_pars.json")) as json_data:
            exp_pars = json.load(json_data)
    
    ## DO
    # Calculate the discharge rate by phenotype
    df = Data.copy()
    df["discharged"] = Data.transfer_to_well_id.isnull() #
    cols = ["transfer_n", "strategy", "phenotype", "discharged"]
    discharge = df[df.discharged & df.phenotype.isin(regular_phenos)][cols].groupby(cols[:-1]).count().unstack(fill_value=0).stack()
    discharge["n"] = df[df.phenotype.isin(regular_phenos)][cols].groupby(cols[:-1]).count().unstack(fill_value=0).stack()
    discharge["tau_out"] = discharge.apply(lambda x: get_tau_out(x, exp_pars), axis=1)
    discharge
    
    # Calculate the admittions rate by phenotype
    df = Data[Data.turnover_strain.isnull()== False].copy() # All Turnover_wells
    cols = ["transfer_n", "strategy", "turnover_strain", "admittions"]
    df["admittions"] = True
    admittions = df[df.turnover_strain.isin(regular_phenos)][cols].groupby(cols[:-1]).count().unstack(fill_value=0).stack()
    admittions["m"] = admittions["admittions"].groupby(level=[0,1]).sum()
    admittions["tau_in"] = admittions.admittions/admittions.m
    admittions
    
    ## SAVE
    discharge.to_pickle(os.path.join(analysis_path, "obj", "04b_turnover_out.pkl"))
    admittions.to_pickle(os.path.join(analysis_path, "obj", "04b_turnover_in.pkl"))

20220412
20220127
20210417


In [3]:
df = Data.copy()
df["discharged"] = Data.transfer_to_well_id.isnull() #
cols = ["transfer_n", "strategy", "phenotype", "discharged"]
discharge = df[df.discharged & df.phenotype.isin(regular_phenos)][cols].groupby(cols[:-1]).count().unstack(fill_value=0).stack()
discharge["n"] = df[df.phenotype.isin(regular_phenos)][cols].groupby(cols[:-1]).count().unstack(fill_value=0).stack()
discharge

discharged    n
transfer_n strategy     phenotype                 
0          Combo        A&B                 0    0
                        AB_r                0    0
                        A_r                 4   20
                        B_r                 4   18
                        S                  44  286
...                                       ...  ...
18         No treatment AB_r              109  109
                        A_r               124  124
                        B_r                19   19
                        S                 104  104
                        UI                 14   14

[672 rows x 2 columns]